<a href="https://colab.research.google.com/github/thyarles/unb-fmc-nlp/blob/main/aula_1/notes_lets_build_the_gpt_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Let's build the GPT Tokenizer

Notes from the video https://www.youtube.com/watch?v=zduSFxRajkE.



* Most of the problem we see on the LLM are from Tokenizers (like do simple math wrong).
* The unicode has three types, UTF-8, UTF-16, and UTF-32. The UTF-8 is the standard because it the only one that has variable length. For latin characters, the UTF-16 add zero word on every letter, and UTF-32 add two zero words.
* We can't use the Unicode to tokenizer because it has a huge code space (about 150 thousand).  

In [ ]:
# To check the UTF-8 value
[ord(x) for x in "Charles."]

[67, 104, 97, 114, 108, 101, 115, 46]

In [ ]:
# To check the UTFs 8, 16 and 32.
print("%s\n%s\n%s" %
(
  list("Charles.".encode("utf-8")),
  list("Charles.".encode("utf-16")),
  list("Charles.".encode("utf-32")))
)

[67, 104, 97, 114, 108, 101, 115, 46]
[255, 254, 67, 0, 104, 0, 97, 0, 114, 0, 108, 0, 101, 0, 115, 0, 46, 0]
[255, 254, 0, 0, 67, 0, 0, 0, 104, 0, 0, 0, 97, 0, 0, 0, 114, 0, 0, 0, 108, 0, 0, 0, 101, 0, 0, 0, 115, 0, 0, 0, 46, 0, 0, 0]


In [5]:
# Byte pair encoding

# Example: aaabdaaabac (vocabulary size = eleven, four tokens)
#          -> find the pair that occurs more frequently and replace it with a single token
#          Z = aa, Y = ab, X = zy -> XdXac (vocabulary size = seven, five tokens)

text = "Charles."                 # any string
tokens = text.encode("utf-8")     # raw bytes
tokens = list(map(int, tokens))   # integers
print("The text has %d chars and %d tokens." % (len(text), len(tokens)))

The text has 8 chars and 8 tokens.
